In [20]:
#!pip install textblob
#!pip install tweepy
#!pip install pycountry
#!pip install wordcloud
#!pip install langdetect
#!pip install better_profanity
#!pip install nbimporter

In [7]:
import re 
import numpy as np
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import matplotlib.pyplot as plt
import pandas as pd
from wordcloud import WordCloud
from better_profanity import profanity

In [8]:
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'

In [25]:
import tweepy
from textblob import TextBlob
import pandas as pd
import time

#Twitter API keys
consumer_key = '3MX6HKSa9Z33xXWF73hK4ureO'
consumer_secret = 'cP1vG4DSSPqcBppOsyZiXd5m34wyNFrpC2M4lqjf3sshQe4znl'
access_token = '39207671-BW75JLnEvwqo3ft00NUEUXJNLJanK5oMgPoyfJWLj'
access_token_secret = 'NFsNkOSPcErlxXBjHXLMTvjP16NaKHSCiEPPYoyJPqa4W'

#Authenticate with the Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#Define a function to search for tweets and perform sentiment analysis
def analyze_tweets(query, since_date, until_date):
    # Create an empty list to store the cleaned tweets
    cleaned_tweets = []
    # Search for tweets with the given query string and date range
    api = tweepy.API(auth)
    tweets = []
    try:
        tweets = api.search_tweets(q=query, count=100, tweet_mode='extended', since_id=since_date, until_id=until_date)
    except tweepy.TweepyException as e:
        print('Error:', e)
        return None

    # Wait for 1 second to avoid hitting the rate limit
    time.sleep(1)

    # Clean the tweets and perform sentiment analysis on each one
    positive_tweets = 0
    negative_tweets = 0
    for tweet in tweets:
        # Extract the full text of the tweet
        if tweet.full_text.startswith('RT'):
            cleaned_tweet = tweet.retweeted_status.full_text
        else:
            cleaned_tweet = tweet.full_text

        # Remove URLs, mentions, and special characters from the tweet text
        cleaned_tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", cleaned_tweet).split())

        # Perform sentiment analysis on the cleaned tweet using TextBlob
        analysis = TextBlob(cleaned_tweet)

        # Count the number of positive and negative tweets
        if analysis.sentiment.polarity > 0:
            positive_tweets += 1
        elif analysis.sentiment.polarity < 0:
            negative_tweets += 1

        # Add the cleaned tweet to the list of cleaned tweets
        cleaned_tweets.append(cleaned_tweet)

    # Calculate the percentage of positive and negative tweets
    total_tweets = len(cleaned_tweets)
    if total_tweets > 0:
        percent_positive = round((positive_tweets / total_tweets) * 100, 2)
        percent_negative = round((negative_tweets / total_tweets) * 100, 2)
    else:
        percent_positive = 0
        percent_negative = 0

    # Create a data frame with the results and return it
    data = {
    'name': query,
    'number of tweets': total_tweets,
    '% positive': percent_positive,
    '% negative': percent_negative,
    'year': since_date.year}
    return pd.DataFrame(data, index=[0])

#Define a list of queries to search for
queries = ['Michael Jordan']

#Define a list of date ranges to search for
date_ranges = [
pd.date_range(start='01/01/2010', end='12/31/2010', freq='M'),
pd.date_range(start='01/01/2011', end='12/31/2011', freq='M'),
pd.date_range(start='01/01/2012', end='12/31/2012', freq='M')
]

#Search for tweets and perform sentiment analysis for each query and date range
results = []
for query in queries:
    for date_range in date_ranges:
        for i in range(len(date_range) - 1):
            since_date = date_range[i]
            until_date = date_range[i + 1]
            result = analyze_tweets(query, since_date, until_date)
            if result is not None:
                results.append(result)

#Concatenate the results into separate data frames for each year
data_frames = {}
for result in results:
    year = result['year'][0]
    if year not in data_frames:
        data_frames[year] = result
    else:
        data_frames[year] = pd.concat([data_frames[year], result], ignore_index=True)

#Print the data frames for each year
for year in data_frames:
    print(f'Data Frame for {year}:')
    print(data_frames[year])
    print()

AttributeError: 'API' object has no attribute 'search_all_tweets'